# Review Produtos - PS Birdie 2020

Primeirante explicando de forma geral, busquei analisar os dados e econtrar possíveis correlações entre os atributos que fosse interessante mostrar. Em seguida, procurei focar em preparar os dados para usar um modelo que seja capaz de prever se uma review é positiva ou não, busquei também extrair palavras que estivessem relacionadas tanto com as review positivas como as negativas, procurei analisar alguma dessas palavra entendendo o porque delas serem tão importantes no contexto.

In [1]:
import pandas as pd
import numpy as np
import string
from nltk import tokenize, FreqDist
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score

product_review = pd.read_csv('dataset/tech_test.tsv', sep='\t')

In [2]:
#Obtendo uma rápida descrição dos dados
product_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20473 entries, 0 to 20472
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   retailer              20473 non-null  object 
 1   category              20473 non-null  object 
 2   breadcrumb            15677 non-null  object 
 3   brand                 20473 non-null  object 
 4   offer_url             20473 non-null  object 
 5   offer_sku             20473 non-null  object 
 6   offer_retailer        20473 non-null  object 
 7   offer_title           20473 non-null  object 
 8   title_keywords        20473 non-null  object 
 9   price                 993 non-null    float64
 10  specs                 20473 non-null  object 
 11  offer_last_update_at  16619 non-null  object 
 12  review_id             20473 non-null  object 
 13  review_title          20356 non-null  object 
 14  review_body           20473 non-null  object 
 15  review_user_rating 

In [3]:
#Visualizando as primeiras 10 instâncias 
pd.set_option('display.max_columns', None)
product_review.head()

,retailer,category,breadcrumb,brand,offer_url,offer_sku,offer_retailer,offer_title,title_keywords,price,specs,offer_last_update_at,review_id,review_title,review_body,review_user_rating,review_posted_at,review_year,review_month,review_week,review_day,review_collected_at,locale,original_offer,variant
0,lowes,Refrigerators,"[""Appliances"", ""Refrigerators"", ""Side-by-Side ...",General Electric,https://www.lowes.com/pd/GE-25-3-cu-ft-Side-by...,1000859768,lowes,GE 25.3-cu ft Side-by-Side Refrigerator with I...,25.3':2 'by':7 'cu':3 'fingerprint':14 'finger...,NaN,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IYNFS'...",2020-05-26 21:51:27.805520,218183104,Functional,Pros: fingerprint resistant so you don't have ...,3.0,2020-03-23,2020,3,13,23,2020-04-24 15:58:56.293182,us,True,"['1000859768', '1000859852']"
1,lowes,Refrigerators,"[""Appliances"", ""Refrigerators"", ""Side-by-Side ...",General Electric,https://www.lowes.com/pd/GE-25-1-cu-ft-Side-by...,1000859852,lowes,GE 25.1-cu ft Side-by-Side Refrigerator with I...,25.1':2 'black':13 'by':7 'cu':3 'ft':4 'ge':1...,NaN,"{'brand': ['GE', 'Ge'], 'model': ['GSS25IBNTS'...",2020-05-26 21:51:28.597592,218183104,Functional,Pros: fingerprint resistant so you don't have ...,3.0,2020-03-23,2020,3,13,23,2020-04-24 14:55:13.485647,us,False,"['1000859768', '1000859852']"
2,lowes,Refrigerators,"[""Appliances"", ""Refrigerators"", ""French Door R...",Frigidaire,https://www.lowes.com/pd/Frigidaire-Gallery-21...,1000289721,lowes,Frigidaire Gallery 21.7-cu ft Counter-depth Fr...,21.7':3 'counter':7 'counter-depth':6 'cu':4 '...,NaN,"{'brand': ['Frigidaire', 'Frigidaire'], 'model...",2020-05-26 21:50:49.940754,190217370,Ample Door Storage User Friendly Visibility,Feels solid and âupscaleâ. Excellent desig...,5.0,2019-09-28,2019,9,39,28,2020-03-30 23:53:02.331711,us,True,['1000289721']
3,bestbuy_us,Refrigerators,"[""Best Buy"", ""Appliances"", ""Refrigerators"", ""B...",Whirlpool,https://www.bestbuy.com/site/whirlpool-21-9-cu...,3928039,bestbuy_us,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,21.9':2 'bottom':6 'bottom-freezer':5 'cu':3 '...,NaN,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",2020-06-05 18:35:55.990040,c407068f-f900-3478-a983-ad74754c1460,So much room,I love this fridge. So much room over having a...,5.0,2019-12-13,2019,12,50,13,2020-04-28 14:09:38.255158,us,True,"['3928039', '3928048', '3979801', '6112639']"
4,bestbuy_us,Refrigerators,"[""Best Buy"", ""Appliances"", ""Refrigerators"", ""B...",Whirlpool,https://www.bestbuy.com/site/whirlpool-21-9-cu...,3979801,bestbuy_us,Whirlpool - 21.9 Cu. Ft. Bottom-Freezer Refrig...,21.9':2 'black':9 'bottom':6 'bottom-freezer':...,NaN,"{'brand': ['whirlpool'], 'Other_UPC': ['883049...",2020-06-05 17:55:04.244327,c407068f-f900-3478-a983-ad74754c1460,So much room,I love this fridge. So much room over having a...,5.0,2019-12-13,2019,12,50,13,2020-04-28 14:09:41.960342,us,False,"['3928039', '3928048', '3979801', '6112639']"


In [4]:
#Quantidade de instâncias e atributos
product_review.shape

(20473, 25)

Analisando os dados, percebi que existem alguns pontos:

- muitas instâncias duplicadas 
- muitos atributos irrelevantes
- instâncias nulas
- combinar atributos

Irei então retirar essas instâncias duplicadas, manter apenas os atributos que considerei realmente relevantes pra classificar se uma review é positiva ou não (possa ser que alguns que irei retirar apresentem relação com o resultado porém não consegui entender qual seria essa possível influência), depois de fazer esses dois procedimentos irei decidir como lidar com as instâncias nulas. Por fim em relação a combinar atributos, transformar os atributos _review_title_ e	_review_body_ em um só facilitará os próximos procedimentos. 

Antes de começar a mexer nos dados queria mostrar a quantidade que cada marca recebeu de pontuação(1 a 5) para seus produtos. Pensei que isso poderia ser interessante para o cliente que talvez queira saber o quanto uma marca é bem avaliada.

In [5]:
pd.crosstab(product_review['brand'], product_review['review_user_rating'])


review_user_rating,1.0,2.0,3.0,4.0,5.0
brand,,,,,
Akdy,0,0,0,0,1
Amana,7,1,9,27,21
Arctic Fresh,0,0,0,1,2
Arctic King,0,0,1,1,2
Avallon,0,0,0,0,1
Avanti,1,0,0,0,0
Bosch,0,3,2,11,28
Cafe,16,8,53,72,154
CafĂŠ,3,0,0,3,1


### Isntancias duplicadas

In [6]:
if (any(product_review['review_id'].duplicated())):              
    product_review = product_review.drop_duplicates('review_id')  #removendo as instâncias duplicadas através do atributo review_id

### Atributos retirados

Os atributos que irei manter serão:

- review_title
- review_body
- review_user_rating

In [7]:
atr_del = ['retailer', 'category', 'breadcrumb', 'brand', 'offer_url', 'offer_sku', 'offer_retailer', 'offer_title', 
           'title_keywords', 'price', 'specs', 'offer_last_update_at', 'review_id', 'review_posted_at', 'review_year',
           'review_month', 'review_week', 'review_day', 'review_collected_at', 'locale', 'original_offer', 'variant']
product_review = product_review.drop(atr_del, axis='columns') 

### Limpando os dados

In [8]:
#Descrição dos dados após retirar as instãncias duplicadas e alguns atributos
product_review.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6284 entries, 0 to 20472
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   review_title        6235 non-null   object 
 1   review_body         6284 non-null   object 
 2   review_user_rating  6284 non-null   float64
dtypes: float64(1), object(2)
memory usage: 196.4+ KB


Dos 3 atributos que se mantiveram, apenas o _review_title_ contém alguns poucos valores nulos, por essa quantidade ser mínima escolhi lidar com essa situação deletando essas instâncias.

In [9]:
product_review = product_review.dropna()

### Combinando dois atributos

In [10]:
product_review['title_body'] = product_review['review_title']+' '+product_review['review_body']

In [11]:
product_review.shape

(6235, 4)

Depois de todos esses procedimentos, a diminuição da quantidade de instâncias foi drástica.

### Rotulando os dados

Irei aplicar futuramente algoritmos de classificação supervisionada, visto isso criei o atributo _positive_ onde 1 será referente a uma review positiva e 0 será negativa. Para isso analisei algumas instâncias do conjunto de dados e notei que os números do atributo _review_user_rating_ de 1-2 estão ligados a uma review negativa, 4-5 estão ligado a uma review positiva, a maior dificuldade foi definir a qual 3 pertenceria, lendo algumas dessas instâncias percebi que a maioria continha mais conteúdo positivo do que negativo por isso defini como 1 também.

In [12]:
classificacao = product_review["review_user_rating"].replace([1,2,3,4,5],[0,0,1,1,1])

In [13]:
product_review["positive"] = classificacao

### StopWords

Removendo as stopwords e pontuações das review

In [14]:
stop_words = stopwords.words('english') + list(string.punctuation)
token_pont = tokenize.WordPunctTokenizer()

review_processed = list()
for review in product_review['title_body']:
    new_review = list()
    words_text = token_pont.tokenize(review)
    for word in words_text:
        if word not in stop_words:
            new_review.append(word)
    review_processed.append(' '.join(new_review))
    
product_review['title_body'] = review_processed

In [15]:
#Verificando se deu certo
product_review['title_body'][0]

'Functional Pros fingerprint resistant constantly wipe Drawers good size plentiful Water dispenser works great Cons noisy making ice point think one thing wrong'

### Vetorizando os dados

Irei utilizar o CountVectorizer para essa etapa pois apresentou resultados melhores do que o TfidfVectorizer.

In [16]:
vectorize = CountVectorizer()
vectorized_data = vectorize.fit_transform(product_review["title_body"])

### Criando um conjunto de testes

Antes de aplicar os modelos, estarei criando um conjunto para o treinamento e um para teste.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(vectorized_data, product_review["positive"], test_size=0.2, random_state=42)

## Aplicando os Modelos 

Os modelos que escolhi foram a Regressão Logística, Máquina de Vetores de Suporte(SVM) e Florestas Aleatórias. Primeiro realizei um GridSearch para encontrar possíveis melhores parâmetros para cada modelo, em seguida eu aplicarei um Ensemble(Soft Voting) tentando ter um resultado que supere todos os classificadores individuais.   

In [18]:
lg_reg = LogisticRegression()
svm_clf = SVC()
rf_clf = RandomForestClassifier()

#Ensemble Hard Voting
voting_clf = VotingClassifier(
    estimators=[('lr', lg_reg), ('rf', rf_clf), ('svc', svm_clf)],
    voting='hard')

### GridSearch

Procurando possíveis melhores parâmetros para os modelos


In [19]:
#Parâmetros da Regressão Logística
param_grid_rlg = {'penalty': ['l1', 'l2'],
                  'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'solver': ['liblinear'],
                  'random_state': [42]}

#Parâmetros SVM
param_grid_svm = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  'gamma':[1,0.1,0.001,0.0001],
                  'kernel':['linear'],
                  'random_state': [42]}

#Parâmetros da RandomForestClassifier
param_grid_rfc = {'bootstrap': [False,True],
                  'n_estimators': [3,10,30],
                  'max_features': [2,4,6,8],
                  'random_state': [42]}

In [20]:
#Aplicando GridSearch nos modelos

grid_search_rlg = GridSearchCV(lg_reg, param_grid_rlg, cv=3)
grid_search_rlg.fit(X_train, y_train)

grid_search_svm = GridSearchCV(svm_clf, param_grid_svm, cv=3)
grid_search_svm.fit(X_train, y_train)

grid_search_rfc = GridSearchCV(rf_clf, param_grid_rfc, cv=3)
grid_search_rfc.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [False, True],
                         'max_features': [2, 4, 6, 8],
                         'n_estimators': [3, 10, 30], 'random_state': [42]})

In [21]:
#Utilizando os melhores parâmetros
lg_reg = grid_search_rlg.best_estimator_
svm_clf = grid_search_svm.best_estimator_
rf_clf = grid_search_rfc.best_estimator_

In [22]:
#Modelos no conjunto de treinamento
for clf in (lg_reg, svm_clf, rf_clf, voting_clf):
    clf.fit(X_train, y_train)
    cross_score = np.mean(cross_val_score(clf, X_train, y_train, cv=3))
    print(cross_score)

0.9290296655047845
0.9214125468328759
0.8907374804594174
0.9192063695364459


Avaliando os modelos usando cross_validation_score, Regressão Logística ganha por muito pouco dos demais em especial do SVM e do Hard Voting, ou seja, a tentiva de usar um Ensemble para obter uma maior pontuação no resultado em relação aos classificadores individualmente foi fracassada, então por isso utilizarei como modelo final a Regressão Logística no conjunto de teste.

In [23]:
#Melhor modelo no conjunto de teste
final_predictions = lg_reg.predict(X_test)
print(accuracy_score(y_test, final_predictions))

0.9414595028067362


### Analisando palavras

Mostrarei algumas das palavras mais influentes tanto em review classificadas como positivas como negativas. 

In [24]:
weights = pd.DataFrame(
    lg_reg.coef_[0].T,
    index = vectorize.get_feature_names()
)

weights.nlargest(20,0)

,0
love,1.096341
great,0.925907
perfect,0.648726
easy,0.646902
quiet,0.605810
well,0.549125
good,0.540051
nice,0.532092
excellent,0.483800
spacious,0.481482


Percepitível que a maiora das palavras nesse "top 20" são referentes a elogios como "great", "perfect", "excellent", algumas podem estar ligadas com sentimentos dos clientes em relação ao produto como "love", "happy", "pleased" e algumas seriam características positivas dos produtos como "quiet", "lots" (imagino que pode estar se refirindo a grande quantidade de coisas que podem ser armazenadas dentro do produto).

In [25]:
weights.nsmallest(20,0)

,0
disappointed,-0.650068
months,-0.624530
junk,-0.563233
horrible,-0.560487
not,-0.544824
already,-0.511042
less,-0.508981
40,-0.467194
buy,-0.460985
worst,-0.459030


Algumas dessas palavras são bem claras em relação a uma review negativa tais como "disappointed", "worst", "junk" porém algumas palavras não consegui entender o porque delas serem tão influentes principalmente números/palavras referentes à tempo/data.